In [4]:
# !pip install scikit-uplift

### Домашнее задание

In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, ClassTransformation, TwoModels

from catboost import CatBoostClassifier

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [9]:
data = pd.read_csv('../lesson_4/materials_lection_4/HW_data.csv')
data.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [10]:
data['offer'].unique()

array(['Buy One Get One', 'No Offer', 'Discount'], dtype=object)

In [11]:
data = data.rename(columns={'offer': 'treatment','conversion': 'target'})

In [12]:
data['treatment'] = data['treatment'].replace({'Buy One Get One': 1, 'Discount': 1, 'No Offer': 0})

In [13]:
print(data.shape)
data.head()

(64000, 9)


,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0


3. сделать разбиение набора данных на тренировочную и тестовую выборки

In [18]:
X = data.drop('target', axis=1)
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
X_train_treatment = X_train['treatment']
X_train = X_train.drop('treatment', axis=1)

X_test_treatment = X_test['treatment']
X_test = X_test.drop('treatment', axis=1)

4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

In [20]:
cat_columns = ['recency', 'zip_code', 'channel']
models_results = {
    'approach': [],
    'uplift@10': [],
    'uplift@20': []
}

5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями

In [21]:
sm = SoloModel(CatBoostClassifier(iterations=20, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, X_train_treatment, estimator_fit_params={'cat_features': cat_columns})

uplift_sm = sm.predict(X_test)

sm_score_at_10 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=X_test_treatment, strategy='by_group', k=.1)
sm_score_at_20 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=X_test_treatment, strategy='by_group', k=.2)

models_results['approach'].append('SoloModel')
models_results['uplift@10'].append(sm_score_at_10)
models_results['uplift@20'].append(sm_score_at_20)

In [23]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, random_state=42, silent=True))
ct = ct.fit(X_train, y_train, X_train_treatment, estimator_fit_params={'cat_features': cat_columns})

uplift_ct = ct.predict(X_test)

ct_score_at_10 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=X_test_treatment, strategy='by_group', k=.1)
ct_score_at_20 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=X_test_treatment, strategy='by_group', k=.2)

models_results['approach'].append('ClassTransformation')
models_results['uplift@10'].append(ct_score_at_10)
models_results['uplift@20'].append(ct_score_at_20)

<ipython-input-23-2f39ac78c6de>:2: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  ct = ct.fit(X_train, y_train, X_train_treatment, estimator_fit_params={'cat_features': cat_columns})


In [25]:
tm = TwoModels(
    estimator_trmnt=CatBoostClassifier(iterations=20, random_state=42, silent=True), 
    estimator_ctrl=CatBoostClassifier(iterations=20, random_state=42, silent=True), 
    method='vanilla'
)
tm = tm.fit(
    X_train, y_train, X_train_treatment,
    estimator_trmnt_fit_params={'cat_features': cat_columns}, 
    estimator_ctrl_fit_params={'cat_features': cat_columns}
)

uplift_tm = tm.predict(X_test)

tm_score_at_10 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=X_test_treatment, strategy='by_group', k=.1)
tm_score_at_20 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=X_test_treatment, strategy='by_group', k=.2)

models_results['approach'].append('TwoModels')
models_results['uplift@10'].append(tm_score_at_10)
models_results['uplift@20'].append(tm_score_at_20)

6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей

In [28]:
models_results = pd.DataFrame(models_results)
models_results

,approach,uplift@10,uplift@20
0,SoloModel,0.105736,0.102581
1,ClassTransformation,0.113567,0.103432
2,TwoModels,0.079869,0.086489


7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево

8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета (конверсии - поле conversion) от значения uplift: 1) сделать прогноз и получить uplift для тестовой выборки 2) отсортировать тестовую выборку по uplift по убыванию 3) разбить на децили (pandas qcut вам в помощь) 4) для каждого дециля посчитать среднюю conversion

9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево